In [2]:
#多层感知机的从0开始实现
import torch
import numpy as np
import d2p

In [4]:
batch_size=256
train_iter,test_iter=d2p.load_data_fashion_mnist(batch_size)
#定义模型参数
num_inputs,num_outputs,num_hiddens=784,10,256
W1=torch.tensor(np.random.normal(0,0.01,(num_inputs,num_hiddens)),dtype=torch.float)
W2=torch.tensor(np.random.normal(0,0.01,(num_hiddens,num_outputs)),dtype=torch.float)
b1=torch.zeros(num_hiddens,dtype=torch.float)
b2=torch.zeros(num_outputs,dtype=torch.float)
params = [W1,b1,W2,b2]
for param in params:
    param.requires_grad_(requires_grad=True)

#定义激活函数
def relu(X):
    return torch.max(input=X,other=torch.tensor(0.0))
#定义模型
def net(X):
    X=X.view((-1,num_inputs))
    H=relu(torch.matmul(X,W1)+b1)
    return torch.matmul(H,W2)+b2
#定义损失函数
loss=torch.nn.CrossEntropyLoss()
#训练模型
num_epochs, lr = 5, 100.0
d2p.train_ch3(net, train_iter, test_iter, loss, num_epochs,batch_size, params, lr)


AttributeError: module 'd2p' has no attribute 'load_data_fashion_mnist'

In [5]:
from torch import nn
#只对权重衰减，不对偏差衰减
def fit_and_plot_pytorch(wd):
    # 对权重参数衰减。权重名称⼀一般是以weight结尾
    net = nn.Linear(num_inputs, 1)
    nn.init.normal_(net.weight, mean=0, std=1)
    nn.init.normal_(net.bias, mean=0, std=1)
    optimizer_w = torch.optim.SGD(params=[net.weight], lr=lr,
    weight_decay=wd) # 对权重参数衰减
    optimizer_b = torch.optim.SGD(params=[net.bias], lr=lr) 
    # 不对偏差参数衰减
    train_ls, ltest_ls = [], []
    for _ in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X), y).mean()
            optimizer_w.zero_grad()
            optimizer_b.zero_grad()
            l.backward()
            # 对两个optimizer实例例分别调⽤用step函数，从⽽而分别更更新权重和偏差
            optimizer_w.step()
            optimizer_b.step()
        train_ls.append(loss(net(train_features),train_labels).mean().item())
        test_ls.append(loss(net(test_features),test_labels).mean().item())
    d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs','loss',range(1, num_epochs + 1), test_ls, ['train','test'])
    print('L2 norm of w:', net.weight.data.norm().item())

In [6]:
#丢弃法
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
import d2p

In [16]:
def dropout(X,drop_prob):
    X=X.float()
    assert 0<=drop_prob<=1
    keep_prob=1-drop_prob
    if keep_prob==0:
        return torch.zeros_like(X)
    randx=torch.rand(X.shape)
    print(randx)
    mask=(randx<keep_prob).float()
    print(mask)
    return mask*X/keep_prob

In [19]:
X=torch.arange(16).view(2,8)
print(dropout(X,0.8))


tensor([[0.3748, 0.9882, 0.7937, 0.8838, 0.3302, 0.0589, 0.7055, 0.4423],
        [0.3338, 0.6837, 0.0489, 0.4526, 0.4407, 0.9516, 0.4510, 0.1424]])
tensor([[0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1.]])
tensor([[ 0.,  0.,  0.,  0.,  0., 25.,  0.,  0.],
        [ 0.,  0., 50.,  0.,  0.,  0.,  0., 75.]])


In [ ]:
#模型构造


In [ ]:
#批量归一化
import time 
import torch
from torch import nn,optim
import sys
import d2p
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def batch_norm(is_training,X,gamma,beta,moving_mean,moving_var,eps,momentum):
    if not is_training:
        X_hat=(X-moving_mean)/torch.sqrt(moving_var+eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape)==2:
            mean=X.mean(dim=0)
            var=((X-mean)**2).mean(dim=0)
        else:
            mean = X.mean(dim=0, keepdim=True).mean(dim=2,keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0,keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            # 训练模式下⽤用当前的均值和⽅方差做标准化
            X_hat = (X - mean) / torch.sqrt(var + eps)
            # 更更新移动平均的均值和⽅方差
            moving_mean = momentum * moving_mean + (1.0 - momentum) *mean
            moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta # 拉伸和偏移
    return Y, moving_mean, moving_var

In [ ]:
#BatchNorm层
class BatchNorm(nn.Module):
    def __init__(self,num_features,num_dims):
        if num_dims==2:#全连接层
            shape=(1,num_features)
        else:
            shape=(1,num_featuresq,1,1)
        self.gamma=nn.Parameter(torch.ones(shape))
        self.beta=nn.Parameter(torch.zeros(shape))
        self.moving_mean=torch.zeros(shape)
        self.moving_var=torch.zeros(shape)
    def forward(self,X):
        if self.moving_mean.device!=X.device:
            self.moving_mean=self.moving_mean.to(X.device)
            self.moving_var=self.moving_var.to(X.device)
        Y,self.moving_mean,self.moving_var=batch_norm(self.training,
                                                     X,self.gamma,self.beta,self.moving_mean,
                                                     selfl.moving_var,eps=1e-5,momentuum=0.9)
        return Y
    